This notebook exists to help perform some transformations and inspections of data.  Goal is to eventually collapse this into a .py file.

Data Sources / credits:
* City and State Zip search: https://pypi.org/project/uszipcode/
* Zip codes from: http://federalgovernmentzipcodes.us/free-zipcode-database-Primary.csv
* Weather station lat/long: https://www1.ncdc.noaa.gov/pub/data/normals/1981-2010/station-inventories/zipcodes-normals-stations.txt


In [1]:
# Install some helpful libraries
#!pip install uszipcode
#!pip install pygeocoder
#!pip install requests

# Importing necessary libraries
import pandas as pd
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)
from math import radians, sin, cos, sqrt, asin

# Setup definition we'll need later
def haversine(lat1, lon1, lat2, lon2):
    R = 6372.8  # Earth radius in kilometers
    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    a = sin(dLat / 2)**2 + cos(lat1) * cos(lat2) * sin(dLon / 2)**2
    c = 2 * asin(sqrt(a))
    return R * c

In [2]:
# Reading the files
fpath = './noaa/mly-tmax-normal.txt'
max_df = pd.read_csv(fpath, delim_whitespace=True, header=None)
fpath = './noaa/mly-tmin-normal.txt'
min_df = pd.read_csv(fpath, delim_whitespace=True, header=None)
fpath = './college-city-mapping.csv'
city_df = pd.read_csv(fpath,header=0)
city_df['min_jan'],city_df['min_feb'],city_df['min_mar'],city_df['min_apr'],city_df['min_may'],city_df['min_jun'],city_df['min_jul'],city_df['min_aug'],city_df['min_sep'],city_df['min_oct'],city_df['min_nov'],city_df['min_dec']='','','','','','','','','','','',''
city_df['max_jan'],city_df['max_feb'],city_df['max_mar'],city_df['max_apr'],city_df['max_may'],city_df['max_jun'],city_df['max_jul'],city_df['max_aug'],city_df['max_sep'],city_df['max_oct'],city_df['max_nov'],city_df['max_dec']='','','','','','','','','','','',''
fpath = './free-zipcode-database-primary.csv'
zip_df = pd.read_csv(fpath,header=0)
fpath = './zipcodes-normals-stations.txt'
station_df = pd.read_fwf(fpath,header=None,converters={1: lambda x: int(x)})
station_df.rename(columns={0:'station_id',1:'zip',2:'city'}, inplace=True)

In [3]:
# strip indicator tag about quality of data, convert to fahrenheit
for h in range(0,max_df.shape[0]):
    for w in range(0,max_df.shape[1]):
        if (len(max_df[w][h])) == 11:
            continue
        else:
            l = len(max_df[w][h])
            max_df[w][h] = float(max_df[w][h][0:l-1])/10

for h in range(0,min_df.shape[0]):
    for w in range(0,min_df.shape[1]):
        if (len(min_df[w][h])) == 11:
            continue
        else:
            l = len(min_df[w][h])
            min_df[w][h] = float(min_df[w][h][0:l-1])/10

max_df.rename(columns={0:'station_id',1:'january',2:'february',3:'march',4:'april',5:'may',6:'june',7:'july',8:'august',9:'september',10:'october',11:'november',12:'december'}, inplace=True)
min_df.rename(columns={0:'station_id',1:'january',2:'february',3:'march',4:'april',5:'may',6:'june',7:'july',8:'august',9:'september',10:'october',11:'november',12:'december'}, inplace=True)

In [48]:
for i in range(city_df.shape[0]):
    print(i)
    city,state = city_df['Geographic Location'][i].split(",")
    result = search.by_city_and_state(city,state)
    sid =[]
    for z in result:
        ctr = 0
        for zm in station_df.zip:
            if int(z.zipcode) == int(zm):
                # yes, we're picking the first one we find
                sid.append(station_df['station_id'][ctr])
                break
            ctr+=1
    if len(sid) == 0:
        print("No station found!")
    else:
        sname = str(sid[0])
        ctr=0
        for sn in max_df.station_id:
            if sname == sn:
                print(min_df.loc[ctr])
                city_df['max_jan'][ctr],city_df['max_feb'][ctr],city_df['max_mar'][ctr],city_df['max_apr'][ctr],city_df['max_may'][ctr],city_df['max_jun'][ctr],city_df['max_jul'][ctr],city_df['max_aug'][ctr],city_df['max_sep'][ctr],city_df['max_oct'][ctr],city_df['max_nov'][ctr],city_df['max_dec'][ctr]=max_df.loc[1],max_df.loc[2],max_df.loc[3],max_df.loc[4],max_df.loc[5],max_df.loc[6],max_df.loc[7],max_df.loc[8],max_df.loc[9],max_df.loc[10],max_df.loc[11],max_df.loc[12]
            ctr+=1
        ctr=0
        for sn in min_df.station_id:
            if sname == sn:
                print(min_df.loc[ctr])
            ctr+=1

0
No station found!
1
No station found!
2
station_id    USC00330061
january              22.2
february             23.8
march                30.9
april                41.7
may                  51.3
june                 60.9
july                 64.8
august               63.5
september            56.6
october              45.7
november               37
december             26.6
Name: 3935, dtype: object
station_id    USC00330061
january              22.2
february             23.8
march                30.9
april                41.7
may                  51.3
june                 60.9
july                 64.8
august               63.5
september            56.6
october              45.7
november               37
december             26.6
Name: 3935, dtype: object
3
station_id    USC00018380
january              34.9
february             38.4
march                45.1
april                51.6
may                    61
june                 68.5
july                 72.1
august              

ValueError: not enough values to unpack (expected 2, got 1)

In [45]:
city_df

College   Geographic Location min_jan min_feb min_mar min_apr  \
0    Abilene Christian           Abilene, TX      20                           
1            Air Force  Colorado Springs, CO                                   
2                Akron             Akron, OH                                   
3              Alabama        Tuscaloosa, AL                                   
4        Alabama State        Montgomery, AL                                   
..                 ...                   ...     ...     ...     ...     ...   
239   Western Michigan         Kalamazoo, MI                                   
240          Wisconsin           Madison, WI                                   
241            Wyoming           Laramie, WY                                   
242               Yale         New Haven, CT                                   
243   Youngstown State        Youngstown, OH                                   

    min_may min_jun min_jul min_aug  ... max_apr max_may max_jun max_jul  \
0                                    ...     NaN     NaN     NaN     NaN   
1                                    ...     NaN     NaN     NaN     NaN   
2                                    ...     NaN     NaN     NaN     NaN   
3                                    ...     NaN     NaN     NaN     NaN   
4                                    ...     NaN     NaN     NaN     NaN   
..      ...     ...     ...     ...  ...     ...     ...     ...     ...   
239                                  ...     NaN     NaN     NaN     NaN   
240                                  ...     NaN     NaN     NaN     NaN   
241                                  ...     NaN     NaN     NaN     NaN   
242                                  ...     NaN     NaN     NaN     NaN   
243                                  ...     NaN     NaN     NaN     NaN   

    max_aug max_sep max_oct max_nov max_dec (0, max_jan)  
0       NaN     NaN     NaN     NaN     NaN          NaN  
1       NaN     NaN     NaN     NaN     NaN          NaN  
2       NaN     NaN     NaN     NaN     NaN          NaN  
3       NaN     NaN     NaN     NaN     NaN          NaN  
4       NaN     NaN     NaN     NaN     NaN          NaN  
..      ...     ...     ...     ...     ...          ...  
239     NaN     NaN     NaN     NaN     NaN          NaN  
240     NaN     NaN     NaN     NaN     NaN          NaN  
241     NaN     NaN     NaN     NaN     NaN          NaN  
242     NaN     NaN     NaN     NaN     NaN          NaN  
243     NaN     NaN     NaN     NaN     NaN          NaN  

[244 rows x 27 columns]

In [47]:
print(city_df['Geographic Location'][1].split())

['Colorado', 'Springs,', 'CO']
